In [1]:
import numpy as np
import pandas as pd
import random
import tqdm
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import seaborn as sns 
import yaml
from pathlib import Path
import sys 
import os 
sys.path.append(os.path.dirname(os.getcwd()))
from util.functions import logistic
from structure_generation.path_connection_gen import ProceduralGraphGenerator, StatsUtils, GraphStructureGenerator




In [2]:
geometric_graph_conf = yaml.safe_load(Path('config.yml').read_text())['reachability']

In [3]:
import multiprocessing, itertools
from typing import Union, Optional, Tuple, List
import networkx as nx 

In [4]:
def run_single_simulation(
    passed_inputs, 
    verbose : bool =  False 
    ):
    """
    Function to run one iteration of the simulation. Need to pass a list instead of arguments to function 
    is so that it can work with pmap.unordered
    """
    mean_degree, structure_name, number_nodes = passed_inputs
    results_dict_irreversable, results_dict_reversable = {}, {}
    if verbose: 
        print(f"simulation run for degree {mean_degree}")
    graphgen = GraphStructureGenerator(
        structure_name=structure_name, 
        num_nodes=number_nodes, 
        target_mean_degree = mean_degree
    )
    graph = graphgen.initial_graph  

    x = ProceduralGraphGenerator(graph)
    infection_matrix_list_irreversable, timesteps_saturation_irreversable, fraction_infected_list_irreversable, info_dict_irreversable = x.infect_till_saturation(
        structure_name = structure_name, modality="irreversable", verbose= False
    )
    infection_matrix_list_reversable, timesteps_saturation_reversable, fraction_infected_list_reversable, info_dict_reversable = x.infect_till_saturation(
        structure_name = structure_name, modality="reversable", verbose= False
    )

    results_dict_irreversable["timesteps_saturation"] = timesteps_saturation_irreversable
    results_dict_irreversable["fraction_infected_list"] = fraction_infected_list_irreversable
    results_dict_irreversable["info_dict"] = info_dict_irreversable

    results_dict_reversable["timesteps_saturation"] = timesteps_saturation_reversable
    results_dict_reversable["fraction_infected_list"] = fraction_infected_list_reversable
    results_dict_reversable["info_dict"] = info_dict_reversable

    return info_dict_reversable, info_dict_irreversable
    
def run_simulation(mean_degree : int, structure_name : str) -> list: 
    simulation_run_reversable, simulation_run_irreversable = [], []
    with multiprocessing.Pool(processes=multiprocessing.cpu_count() * 2 - 1) as p:
        iterThis = itertools.repeat(
            [
                mean_degree, 
                structure_name,
                geometric_graph_conf['nodes']
            ],
            geometric_graph_conf['num_simulation_runs'],
        )
        with tqdm.tqdm(total=geometric_graph_conf['num_simulation_runs']) as pbar:
            for _ in p.imap_unordered(run_single_simulation, iterThis):
                pbar.update()
                simulation_run_reversable.append(_[0])
                simulation_run_irreversable.append(_[1])

    final_dict_reversable = {k : [] for k in list(_[0].keys())}
    for key in final_dict_reversable.keys():
        for simulation_dict in simulation_run_reversable: 
            final_dict_reversable[key].append(simulation_dict[key])

    final_dict_irreversable = {k : [] for k in list(_[0].keys())}
    for key in final_dict_irreversable.keys():
        for simulation_dict in simulation_run_irreversable: 
            final_dict_irreversable[key].append(simulation_dict[key])

    return final_dict_reversable, final_dict_irreversable


def plot_results(results_dict : dict, structure_name : str ):
    for iter in results_dict['fraction_infected_list']:
        #print(iter['info_dict'])
        timesteps_list = [x for x in range(len(iter))]
        #try:
        #    p, cov = curve_fit(logistic, timesteps_list, iter)
        #    logistic_curve_data = logistic(timesteps_list, *p)
        #except:
        #    pass
        plt.plot(timesteps_list, iter, )#label = f"average simulated saturation across {len(result['timesteps_saturation'])} runs")
    #plt.plot(timesteps_list, logistic_curve_data, label = "fitted logistic")
    plt.title(
        f"saturation curve for {structure_name}     modality: {results_dict['info_dict'][0]['modality']}    mean degree: {round(results_dict['info_dict'][0]['average_degree'], 2)}")
    
    plt.xlabel("number simulation timesteps")
    plt.ylabel("fraction of giant graph infected")
    plt.legend()
    plt.show()

REVERSABLE GEOMETRIC GRAPH SIMULATION

In [5]:
graph_name = "random_geometric"
for degree in geometric_graph_conf['mean_degrees']: 
    print(f"simulation run for mean degree {degree}")
    simulation_run = run_simulation(degree, graph_name)
    
    for mode in simulation_run: 
        print(mode['info_dict'][:5])
        plot_results(mode, graph_name)

simulation run for mean degree 5


100%|██████████| 30/30 [00:31<00:00,  1.05s/it]


AttributeError: 'tuple' object has no attribute 'keys'

In [ ]:
degrees_list = []
for run in simulation_run['info_dict']:
    degrees_list.extend(run['degree_list'])

sns.displot(degrees_list)
plt.title("degree distribution plot for random geometric graph")
plt.xlabel("degree")
plt.ylabel("count density")

IRREVERSABLE GEOMETRIC GRAPH SIMULATION

In [ ]:
"""graph_name = "random_geometric"
for degree in conf['mean_degrees']: 
    print(f"simulation run for mean degree {degree}")
    simulation_run = run_simulation("irreversable",degree, graph_name)
    print(simulation_run['info_dict'][:5])
    plot_results(simulation_run, graph_name)"""

Results for reversable albert simulation 

In [ ]:
graph_name = "barabasi_albert"
for degree in conf['mean_degrees']: 
    print(f"simulation run for mean degree {degree}")
    simulation_run = run_simulation("reversable",degree, graph_name)
    print(simulation_run['info_dict'][:30])
    plot_results(simulation_run, graph_name)

In [ ]:
simulation_put = []
removed_len = []
for run in simulation_run['fraction_infected_list']:
    first_val = run[0]
    runer = [x for x in run if x > 0.03]
    removed_len = len(run) - len(runer)
    #simulation_put.append(runer)
    print(removed_len)
    removed_len.append(removed_len)

In [ ]:


def plot_results(lister, structure_name : str ):
    for iter in lister:
        #print(iter['info_dict'])
        timesteps_list = [x for x in range(len(iter))]
        #try:
        #    p, cov = curve_fit(logistic, timesteps_list, iter)
        #    logistic_curve_data = logistic(timesteps_list, *p)
        #except:
        #    pass
        plt.plot(timesteps_list, iter, )#label = f"average simulated saturation across {len(result['timesteps_saturation'])} runs")
    #plt.plot(timesteps_list, logistic_curve_data, label = "fitted logistic")
    #plt.title(
    #    f"saturation curve for {structure_name}     modality: {results_dict['info_dict'][0]['modality']}    mean degree: {round(results_dict['info_dict'][0]['average_degree'], 2)}")
    
    plt.xlabel("number simulation timesteps")
    plt.ylabel("fraction of giant graph infected")
    plt.legend()
    plt.show()

In [ ]:
plot_results(simulation_put, "foo")

In [ ]:
degrees_list = []
for run in simulation_run['info_dict']:
    degrees_list.extend(run['degree_list'])

sns.displot(degrees_list)
plt.title("degree distribution plot for barabassi albert graph")
plt.xlabel("degree")
plt.ylabel("count density")

Results for irreversable albert simulation 

In [ ]:
graph_name = "barabasi_albert"
for degree in conf['mean_degrees']: 
    print(f"simulation run for mean degree {degree}")
    simulation_run = run_simulation("irreversable",degree, graph_name)
    print(simulation_run['info_dict'][:30])
    plot_results(simulation_run, graph_name)